<a href="https://colab.research.google.com/github/imehr/youtube-framer/blob/main/youGPTube_Single_File_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive')

# Path to the file containing the API key
file_path = '/content/drive/MyDrive/For Google Colab/mohem.txt'

# Error handling for file reading
try:
    with open(file_path, 'r') as f:
        api_key = f.read().strip()
except FileNotFoundError:
    print(f"File not found at {file_path}")
    # Exit or handle error accordingly
    raise

# Check and set API key
if api_key:
    os.environ['OPENAI_API_KEY'] = api_key
else:
    print("API key is empty or invalid")
    # Exit or handle error accordingly
    raise ValueError("Invalid API key")


Mounted at /content/drive


In [4]:
print(file_path)  # to confirm the path is correct
print(api_key[:5])  # print the first 5 characters of your API key to ensure it's being read correctly


/content/drive/MyDrive/For Google Colab/mohem.txt
sk-rt


# YouGPTube 🦾

## TL;DR 👇

* Summarize any YouTube video using whisper and chatGPT

## How it works 🤔

![yougptube](https://user-images.githubusercontent.com/18450628/229377710-95fb8645-3d71-47d0-b3ba-0fd05941b083.png)

Here are the main steps:

1) Extract the audio using youtube-dl
2) Process the audio into smaller chunks
3) Each chunk is transcribed using whisper, OpenAI's powerful speech2text model
4) Each transcription is summarized using ChatGPT

## Imports and dependencies️ ⚙️

In [5]:
!pip install openai
!pip install -U yt-dlp

import yt_dlp
import os
import shutil
import librosa
import openai
import soundfile as sf
#from youtube_dl.utils import DownloadError

# Assuming the API key is set as an environment variable outside the script
api_key = os.getenv("OPENAI_API_KEY")
assert api_key is not None, "Set your OpenAI API key"

# Alternatively, read the API key from a file (more secure than hardcoding)
# Ensure the file is located in a secure and restricted directory
api_key_file_path = '/content/drive/MyDrive/For Google Colab/mohem.txt'  # Replace with your actual path

try:
    with open(api_key_file_path, 'r') as f:
        api_key = f.read().strip()
    os.environ['OPENAI_API_KEY'] = api_key
except FileNotFoundError:
    print(f"API key file not found at {api_key_file_path}")
    # Handle error or exit
    raise

# The rest of your code...



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.4 MB/s eta 0:00:00


## Utility functions 🔋

In [6]:
def find_audio_files(path, extension=".mp3"):
    """Recursively find all files with extension in path."""
    audio_files = []
    for root, dirs, files in os.walk(path):
        for f in files:
            if f.endswith(extension):
                audio_files.append(os.path.join(root, f))

    return audio_files

## Download youtube audio 🔈

In [7]:
!pip install -U yt-dlp  # Installing yt-dlp

from yt_dlp import YoutubeDL
import os

def youtube_to_mp3(youtube_url: str, output_dir: str) -> (str, str):
    """Download the audio from a youtube video, save it to output_dir as an .mp3 file."""
    audio_filename = None
    video_title = None

    # config
    ydl_config = {
        "format": "bestaudio/best",
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
        "outtmpl": os.path.join(output_dir, "%(title)s.%(ext)s"),
    }

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with YoutubeDL(ydl_config) as ydl:
            info_dict = ydl.extract_info(youtube_url, download=False)
            video_title = info_dict.get('title', 'Unknown Title')
            ydl.download([youtube_url])
            audio_filename = find_audio_files(output_dir)[0]  # Assuming `find_audio_files` is defined elsewhere
    except Exception as e:  # Catch all exceptions, as yt_dlp may throw various types
        print(f"An error occurred: {e}")

    return audio_filename, video_title






## Chunk the audio 🍪

Chunking is necessary in the case where we have very long audio files, since both whisper and ChatGPT have limits of how much audio/text you can process in one go.
It is not necessary for shorter videos.

In [8]:
def chunk_audio(filename, segment_length: int, output_dir):
    """segment lenght is in seconds"""

    print(f"Chunking audio to {segment_length} second segments...")

    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    # load audio file
    audio, sr = librosa.load(filename, sr=44100)

    # calculate duration in seconds
    duration = librosa.get_duration(y=audio, sr=sr)

    # calculate number of segments
    num_segments = int(duration / segment_length) + 1

    print(f"Chunking {num_segments} chunks...")

    # iterate through segments and save them
    for i in range(num_segments):
        start = i * segment_length * sr
        end = (i + 1) * segment_length * sr
        segment = audio[start:end]
        sf.write(os.path.join(output_dir, f"segment_{i}.mp3"), segment, sr)

    chunked_audio_files = find_audio_files(output_dir)
    return sorted(chunked_audio_files)

## Speech2text 🗣

Here we use OpenAI's whisper model to transcribe audio files to text.

In [9]:
def transcribe_audio(audio_files: list, output_file=None, model="whisper-1") -> list:

    print("converting audio to text...")

    transcripts = []
    for audio_file in audio_files:
        audio = open(audio_file, "rb")
        response = openai.Audio.transcribe(model, audio)
        transcripts.append(response["text"])

    if output_file is not None:
        # save all transcripts to a .txt file
        with open(output_file, "w") as file:
            for transcript in transcripts:
                file.write(transcript + "\n")

    return transcripts

## Summarize 📝

Here we ask chatGPT to take the raw transcripts and transcribe them for us to short bullet points.

In [10]:
def summarize(
    chunks: list[str], system_prompt: str, model="gpt-3.5-turbo", output_file=None
):

    print(f"Summarizing with {model=}")

    summaries = []
    for chunk in chunks:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": chunk},
            ],
        )
        summary = response["choices"][0]["message"]["content"]
        summaries.append(summary)

    if output_file is not None:
        # save all transcripts to a .txt file
        with open(output_file, "w") as file:
            for summary in summaries:
                file.write(summary + "\n")

    return summaries

## Putting it all together 🍱

In [11]:
def summarize_youtube_video(youtube_url, outputs_dir):
    raw_audio_dir = f"{outputs_dir}/raw_audio/"
    chunks_dir = f"{outputs_dir}/chunks"
    transcripts_file = f"{outputs_dir}/transcripts.txt"
    summary_file = f"{outputs_dir}/summary.txt"
    segment_length = 10 * 60  # chunk to 10 minute segments

    if os.path.exists(outputs_dir):
        # delete the outputs_dir folder and start from scratch
        shutil.rmtree(outputs_dir)
        os.mkdir(outputs_dir)

    # download the video using youtube-dl
    audio_filename, video_title = youtube_to_mp3(youtube_url, output_dir=raw_audio_dir)


    # chunk each audio file to shorter audio files (not necessary for shorter videos...)
    chunked_audio_files = chunk_audio(
        audio_filename, segment_length=segment_length, output_dir=chunks_dir
    )

    # transcribe each chunked audio file using whisper speech2text
    transcriptions = transcribe_audio(chunked_audio_files, transcripts_file)

    # summarize each transcription using chatGPT
    system_prompt = """
    You are a helpful assistant that summarizes youtube videos.
    You are provided chunks of raw audio that were transcribed from the video's audio.
    Summarize the current chunk to succint and clear bullet points of its contents.
    """
    summaries = summarize(
        transcriptions, system_prompt=system_prompt, output_file=summary_file
    )

    system_prompt_tldr = """
    You are a helpful assistant that summarizes youtube videos.
    Someone has already summarized the video to key points.
    Summarize the key points to one or two sentences that capture the essence of the video.
    """
    # put the entire summary to a single entry
    long_summary = "\n".join(summaries)
    short_summary = summarize(
        [long_summary], system_prompt=system_prompt_tldr, output_file=summary_file
    )[0]

    return long_summary, short_summary, video_title

In [12]:
# Function to summarize a single video
def summarize_single_video(youtube_url, outputs_dir):
    audio_filename, video_title = youtube_to_mp3(youtube_url, output_dir=outputs_dir)
    long_summary, short_summary = summarize_youtube_video(audio_filename, outputs_dir)  # Assuming summarize_youtube_video is defined elsewhere
    save_to_markdown_file(video_title, youtube_url, long_summary, short_summary, outputs_dir)

In [ ]:
# Your existing imports and configurations
import openai

openai.api_key = api_key

# Function to save markdown
def save_to_markdown_file(video_title, youtube_url, long_summary, short_summary, output_dir):
    # Sanitize the video title to create a valid filename
    sanitized_title = ''.join(e for e in video_title if e.isalnum() or e == ' ').replace(' ', '_')
    markdown_filename = f"{output_dir}/{sanitized_title}.md"

    # Create or overwrite the markdown file
    with open(markdown_filename, 'w') as f:
        f.write(f"# {video_title}\n\n")
        f.write(f"## YouTube URL\n{youtube_url}\n\n")
        f.write("## Long Summary\n")
        f.write(long_summary)
        f.write("\n\n")
        f.write("## TL;DR\n")
        f.write(short_summary)

# Your code to generate summaries
youtube_url = "https://www.youtube.com/watch?v=KH0XNW0t6-k"
outputs_dir = "outputs/"

long_summary, short_summary, video_title = summarize_youtube_video(youtube_url, outputs_dir)

# Your code to display summaries
print("Summaries:")
print("=" * 80)
print("Long summary:")
print("=" * 80)
print(long_summary)
print()

print("=" * 80)
print("Video - TL;DR")
print("=" * 80)
print(short_summary)

# Add this line to save the summaries to a markdown file
# Replace 'Your Video Title' with the actual video title
# video_title = 'Your Video Title'
save_to_markdown_file(video_title, youtube_url, long_summary, short_summary, outputs_dir)
